## Install Packages

In [17]:
%pip install tiktoken


     ---------------------------------------- 0.0/579.4 kB ? eta -:--:--
      --------------------------------------- 10.2/579.4 kB ? eta -:--:--
     - ----------------------------------- 30.7/579.4 kB 330.3 kB/s eta 0:00:02
     -- ---------------------------------- 41.0/579.4 kB 281.8 kB/s eta 0:00:02
     ---- -------------------------------- 71.7/579.4 kB 393.8 kB/s eta 0:00:02
     ------- ---------------------------- 122.9/579.4 kB 554.9 kB/s eta 0:00:01
     -------------- --------------------- 235.5/579.4 kB 850.1 kB/s eta 0:00:01
     ------------------------- ------------ 389.1/579.4 kB 1.2 MB/s eta 0:00:01
     ------------------------------------ - 553.0/579.4 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 579.4/579.4 kB 1.5 MB/s eta 0:00:00


In [2]:
%pip install unstructured openai langchain unstructured[local-inference] pinecone-client

  Using cached pinecone_client-2.2.1-py3-none-any.whl (177 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.0/60.0 kB 456.1 kB/s eta 0:00:00
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
  Using cached layoutparser-0.3.4-py3-none-any.whl (19.2 MB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started


## Imports

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate,)
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from langchain.document_loaders import DirectoryLoader
import glob
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

c:\Users\juantello\Anaconda3\envs\langchain\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
os.environ['PINECONE_API_KEY'] = "42e3e74e-a552-42c9-ad4b-f6413a6697a8"
os.environ['PINECONE_API_ENV'] = "us-west4-gcp"
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


## List files in directory

In [6]:
folder_path = "ICOLD - CFRD New Bulletin 2023/"
files = glob.glob(folder_path + "**/*.pdf")

In [7]:
files

['ICOLD - CFRD New Bulletin 2023\\Dam Response\\(ASCE)1090-0241(2006)132_6(778).pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\(ASCE)GT.1943-5606.0000021.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\(ASCE)GT.1943-5606.0001501.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\feng.2011.164.1.33.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\jdare.17.00011.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\jgeen.17.00142.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\jgeot.17.p.095.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\Monitoring System.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dam Response\\sdar.64119.381.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dynamic Response\\(ASCE)0733-9410(1995)121_2(185).pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dynamic Response\\(ASCE)1090-0241(2004)130_1(103).pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dynamic Response\\1-s2.0-S0141029619335564-main.pdf',
 'ICOLD - CFRD New Bulletin 2023\\Dynamic Response\\1-s2.0-S026


### Pick a File To Query

In [11]:
def get_title(pdf_file):
    # Split the document into sentences
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
    # llm = ChatOpenAI
    loader = UnstructuredPDFLoader(pdf_file)
    data = loader.load()
    head = data[0].page_content[:400]
    prompt = "What is the title of the paper that begins with: " + head + "Respond only with the title."
    template="You are a helpful assistant that guesses the title of a paper."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template="{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
    chain = LLMChain(llm=ChatOpenAI(temperature=0, max_tokens=800), prompt=chat_prompt)
    return chain.run(prompt), data


In [12]:
texts = RecursiveCharacterTextSplitter(data)


NameError: name 'data' is not defined

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=0)


In [30]:
df = pd.read_csv("summary.csv")

In [31]:
df.file_path = df.file_path.str.replace('/','\\')

In [32]:
df.file_path.iloc[0]

'ICOLD - CFRD New Bulletin 2023\\Structural Analysis\\ICOLD - CFRD New Bulletin 2023\\Structural Analysis\\Behaviour of the reinforced concrete face slabs of concrete faced rockfill dams during impounding.pdf'

In [38]:
df = df.groupby("file_path").first()

In [39]:
df.to_csv("summary.csv", index=False)

In [34]:
import pandas as pd
csv_file = 'summary.csv'

# Check if the file exists
if os.path.exists(csv_file): 
    # If it exists, load the data from the file into a Pandas dataframe
    df = pd.read_csv(csv_file)
import random
for pdf_file in files:
    if not pdf_file in list(df.file_path):
        loader = UnstructuredPDFLoader(pdf_file)
        print(f"Loading {pdf_file.split(os.sep)[-1]}" )
        data = loader.load()
        print (f'You have {len(data)} document(s) in your data')
        print (f'There are {len(data[0].page_content)} characters in your document')

        title, data = get_title(pdf_file)
        # Split the text
        texts = text_splitter.split_documents(data)        ## Summarize
        folder = pdf_file.split(os.sep)[-2]
        # Build the chain
        llm = ChatOpenAI(temperature=0., max_tokens=400)
        chain = load_qa_chain(llm, chain_type="map_reduce", verbose=True)
        summary = chain.run(input_documents=texts, question='summarize')
        summary
        # Define the data to append
        new_data = {'file_path': pdf_file, 'title': title, 'summary': summary}

        # Check if the file exists
        if os.path.exists(csv_file): 
            # If it exists, load the data from the file into a Pandas dataframe
            df = pd.read_csv(csv_file)
            # Append the new data to the existing dataframe
            df = df.append(new_data, ignore_index=True)
        else:
            # If it does not exist, create a new dataframe with the first summary
            df = pd.DataFrame([new_data])

    # Write the updated data to the file
    df.to_csv(csv_file, index=False)
    print(df.head)

KeyboardInterrupt: 